In [1]:
import pandas as pd
from scipy.io import arff
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

# 数据处理

In [63]:
#读取arff文件
data, _ = arff.loadarff("E:\data\multiclass\multiclass\data15 Sampled Scenarios.csv.arff")
df = pd.DataFrame(data)
l=len(df['marker'])
# #去掉重复值，每个重复值有多少
# set(df['marker'])
# df['marker'].value_counts()
for i in range(l):
    df['R4-PA:Z'][i] = float(df['R4-PA:Z'][i])

    df['R3-PA:Z'][i] = float(df['R3-PA:Z'][i])

    df['R2-PA:Z'][i] = float(df['R2-PA:Z'][i])

    df['R1-PA:Z'][i] = float(df['R1-PA:Z'][i])
    
    df['marker'][i] = int(df['marker'][i].decode('UTF-8'))

n=0
for i in df['marker']:
    if i == 41:
        df['marker'][n]=0
    elif i >= 7 and i <= 12:
        df['marker'][n]=2
    elif i >=15 and i <= 20:
        df['marker'][n]=3
    elif i >=21 and i <= 40:
        df['marker'][n]=4
    else:
        df['marker'][n]=1
    n=n+1
    
print(df['marker'].value_counts())

#删除空值和inf所在行
df=df[~df.isin([np.nan, np.inf, -np.inf]).any(1)]
#保存数据
df.to_csv("E:\data\multiclass\multiclass\data15.15.csv",index=0)

E:\ProgramData\Anaconda3\envs\py37tf\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == "__main__":
E:\ProgramData\Anaconda3\envs\py37tf\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
E:\ProgramData\Anaconda3\envs\py37tf\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

4    2241
1    1347
3     739
0     514
2     435
Name: marker, dtype: int64


In [7]:
#转化成字符串
df['marker'][1].decode('UTF-8')

'41'

In [36]:
df['marker'].value_counts()

4    2845
1     927
2     569
3     452
0     173
Name: marker, dtype: int64

# CNN

In [5]:
df=pd.read_csv("E:\data\multiclass\multiclass\data1.1.csv")
ndarray = df.values
#把特征和标签分开用于归一化
label=ndarray[:,-1].reshape(4617,1)
feature=ndarray[:,:-1]
#标签变成独热编码
import tensorflow as tf
label = tf.keras.utils.to_categorical(label)
#数据归一化
minmax_scale=preprocessing.MinMaxScaler(feature_range=(0,1))
scalefeature=minmax_scale.fit_transform(feature)
#先将数据集进行拼接，要不然我们只针对样本进行采样的话，会找不到对应的标签的
data = np.hstack((feature,label))
#使用随机采样方式划分数据集
train_set,test_set = train_test_split(data,test_size = 0.2,random_state = 42)
#把训练和测试的特征和标签分开
tr_feature=train_set[:,:128]
tr_label=train_set[:,128:134]
te_feature=test_set[:,:128]
te_label=test_set[:,128:134]
tr_feature=tr_feature.astype(np.float32)
tr_label=tr_label.astype(np.float32)
#扩充特征的维度否则会报错
tr_feature=np.expand_dims(tr_feature,axis=2)
te_feature=np.expand_dims(te_feature,axis=2)

In [6]:
df

,R1-PA1:VH,R1-PM1:V,R1-PA2:VH,R1-PM2:V,R1-PA3:VH,R1-PM3:V,R1-PA4:IH,R1-PM4:I,R1-PA5:IH,R1-PM5:I,...,control_panel_log4,relay1_log,relay2_log,relay3_log,relay4_log,snort_log1,snort_log2,snort_log3,snort_log4,marker
0,70.399324,127673.0908,-49.572308,127648.0176,-169.578319,127723.2374,65.689611,605.91099,-57.003571,626.78553,...,0,0,0,0,0,0,0,0,0,0
1,73.688102,130280.7109,-46.300719,130255.6377,-166.278082,130355.9307,71.831719,483.59351,-50.947407,500.98896,...,0,0,0,0,0,0,0,0,0,0
2,73.733939,130305.7842,-46.254883,130280.7109,-166.232245,130381.0040,71.808800,483.59351,-50.913030,500.98896,...,0,0,0,0,0,0,0,0,0,0
3,74.083443,130581.5902,-45.899649,130556.5169,-165.882741,130656.8100,72.152575,482.86107,-50.437475,499.15786,...,0,0,0,0,0,0,0,0,0,0
4,74.553268,131083.0556,-45.424094,131057.9823,-165.424375,131158.2754,72.118198,484.50906,-50.013486,497.69298,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4612,28.246819,130807.2496,-91.713354,130782.1763,148.287207,130882.4694,25.760182,458.14122,-95.494876,466.01495,...,0,0,0,0,0,0,0,0,0,1
4613,28.229631,130882.4694,-91.730543,130882.4694,148.264289,130957.6892,25.553918,459.42299,-95.683952,466.93050,...,0,0,0,0,0,0,0,0,0,1
4614,28.195253,130957.6892,-91.770650,130932.6159,148.224182,131032.9090,25.267439,460.70476,-95.752707,467.66294,...,0,0,0,0,0,0,0,0,0,1
4615,28.000447,131057.9823,-91.971185,131057.9823,148.035106,131133.2021,24.316329,464.36696,-96.251180,468.94471,...,0,0,0,0,0,0,0,0,0,1


In [10]:
label

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])

In [29]:
scalefeature

array([[0.6955546 , 0.85669456, 0.36231815, ..., 0.        , 0.        ,
        0.        ],
       [0.70469051, 0.87845188, 0.37140674, ..., 0.        , 0.        ,
        0.        ],
       [0.70481784, 0.87866109, 0.37153408, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.57831575, 0.88410042, 0.24508961, ..., 0.        , 0.        ,
        0.        ],
       [0.57777459, 0.88493724, 0.24453252, ..., 0.        , 0.        ,
        0.        ],
       [0.57769502, 0.88514644, 0.24446885, ..., 0.        , 0.        ,
        0.        ]])

In [20]:
model = Sequential()
model.add(Conv1D(256, 5, activation='tanh', input_shape=(tr_feature.shape[1], 1)))
model.add(Dropout(0.1))
model.add(Conv1D(128, 3, activation='tanh'))
model.add(MaxPooling1D(3))
#model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='tanh'))
model.add(Dropout(0.1))
model.add(Conv1D(50, 3, activation='tanh'))
model.add(Dropout(0.1))
model.add(Conv1D(10, 3, activation='tanh'))
model.add(Flatten())
model.add(Dense(160, activation='tanh'))
model.add(Dense(5, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 124, 256)          1536      
_________________________________________________________________
dropout (Dropout)            (None, 124, 256)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 122, 128)          98432     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 40, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 38, 128)           49280     
_________________________________________________________________
dropout_1 (Dropout)          (None, 38, 128)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 36, 50)            1

In [21]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
reduceLr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=0, mode='auto', cooldown=0, min_lr=0.001)

train_history=model.fit(x=tr_feature, 
                        y=tr_label,
                    
                        validation_split=0.1,
                        epochs=200,
                        batch_size=20,
                        callbacks=[reduceLr])

Train on 3323 samples, validate on 370 samples
Epoch 1/200
3323/3323 [==============================] - 8s 2ms/sample - loss: 1.2342 - accuracy: 0.5808 - val_loss: 1.2465 - val_accuracy: 0.5676
Epoch 2/200
3323/3323 [==============================] - 6s 2ms/sample - loss: 1.1663 - accuracy: 0.5832 - val_loss: 1.1763 - val_accuracy: 0.5676
Epoch 3/200
3323/3323 [==============================] - 6s 2ms/sample - loss: 1.0978 - accuracy: 0.5832 - val_loss: 1.1017 - val_accuracy: 0.5676
Epoch 4/200
3323/3323 [==============================] - 6s 2ms/sample - loss: 1.0709 - accuracy: 0.5832 - val_loss: 1.0780 - val_accuracy: 0.5676
Epoch 5/200
3323/3323 [==============================] - 6s 2ms/sample - loss: 1.0448 - accuracy: 0.5835 - val_loss: 1.0342 - val_accuracy: 0.5676
Epoch 6/200
3323/3323 [==============================] - 6s 2ms/sample - loss: 1.0230 - accuracy: 0.5844 - val_loss: 1.0316 - val_accuracy: 0.5676
Epoch 7/200
3323/3323 [==============================] - 6s 2ms/sample 

3323/3323 [==============================] - 5s 2ms/sample - loss: 0.5356 - accuracy: 0.7668 - val_loss: 0.7094 - val_accuracy: 0.7108
Epoch 112/200
3323/3323 [==============================] - 5s 2ms/sample - loss: 0.5220 - accuracy: 0.7776 - val_loss: 0.7435 - val_accuracy: 0.7135
Epoch 113/200
3323/3323 [==============================] - 5s 2ms/sample - loss: 0.5160 - accuracy: 0.7755 - val_loss: 0.7147 - val_accuracy: 0.6784
Epoch 114/200
3323/3323 [==============================] - 5s 2ms/sample - loss: 0.5122 - accuracy: 0.7761 - val_loss: 0.7172 - val_accuracy: 0.7108
Epoch 115/200
3323/3323 [==============================] - 5s 2ms/sample - loss: 0.5050 - accuracy: 0.7806 - val_loss: 0.6968 - val_accuracy: 0.7243
Epoch 116/200
3323/3323 [==============================] - 6s 2ms/sample - loss: 0.5020 - accuracy: 0.7842 - val_loss: 0.7350 - val_accuracy: 0.7054
Epoch 117/200
3323/3323 [==============================] - 5s 2ms/sample - loss: 0.5006 - accuracy: 0.7924 - val_loss: 0

3323/3323 [==============================] - 6s 2ms/sample - loss: 0.4974 - accuracy: 0.7815 - val_loss: 0.7363 - val_accuracy: 0.7189
Epoch 167/200
3323/3323 [==============================] - 6s 2ms/sample - loss: 0.4610 - accuracy: 0.8005 - val_loss: 0.7163 - val_accuracy: 0.7541
Epoch 168/200
3323/3323 [==============================] - 6s 2ms/sample - loss: 0.4570 - accuracy: 0.8047 - val_loss: 0.6882 - val_accuracy: 0.7486
Epoch 169/200
3323/3323 [==============================] - 6s 2ms/sample - loss: 0.4358 - accuracy: 0.8113 - val_loss: 0.6977 - val_accuracy: 0.7162
Epoch 170/200
3323/3323 [==============================] - 5s 2ms/sample - loss: 0.4420 - accuracy: 0.8119 - val_loss: 0.6438 - val_accuracy: 0.7622
Epoch 171/200
3323/3323 [==============================] - 5s 2ms/sample - loss: 0.4666 - accuracy: 0.7933 - val_loss: 0.7041 - val_accuracy: 0.7568
Epoch 172/200
3323/3323 [==============================] - 6s 2ms/sample - loss: 0.4377 - accuracy: 0.8143 - val_loss: 0

In [2]:
import tensorflow as tf
import time
import pandas as pd
import os
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Dropout, Flatten, Dense, \
    GlobalAveragePooling2D
from tensorflow.keras import Model
from matplotlib import pyplot as plt
from collections import Counter
from imblearn.over_sampling import BorderlineSMOTE
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [31]:
all_df=pd.read_csv("E:\data\multiclass\multiclass\data13.1.csv")
tr_Y = all_df['marker']
tr_X = all_df.drop('marker', axis=1)
sm = BorderlineSMOTE(random_state=42,kind="borderline-1")
tr_X, tr_Y = sm.fit_resample(tr_X, tr_Y)
all_df=pd.concat([tr_X,tr_Y],axis=1)
ndarray = all_df.values
#把特征和标签分开用于归一化
label=ndarray[:,-1].reshape(ndarray.shape[0],1)
feature=ndarray[:,:-1]
from sklearn import preprocessing
#数据归一化
minmax_scale=preprocessing.MinMaxScaler(feature_range=(0,1))
scalefeature=minmax_scale.fit_transform(feature)
#先将数据集进行拼接，要不然我们只针对样本进行采样的话，会找不到对应的标签的
data = np.hstack((scalefeature,label))
#使用随机采样方式划分数据集
train_set,test_set = train_test_split(data,test_size = 0.2,random_state = 42)
n=len(train_set)
m=len(test_set)
#把训练和测试的特征和标签分开
tr_feature=train_set[:,:-1]
tr_label=train_set[:,-1].reshape(n,1)
te_feature=test_set[:,:-1]
te_label=test_set[:,-1].reshape(m,1)

b=np.zeros(tr_feature.shape[0]*16).reshape(tr_feature.shape[0],16)
a=np.hstack((tr_feature,b)).reshape(tr_feature.shape[0],12,12)
# 给数据增加一个维度，使数据和网络结构匹配
x_train = a.reshape(a.shape[0], 12, 12, 1)
#x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print("x_train.shape", x_train.shape)
tr_label=tr_label.astype(np.uint8)
b=np.zeros(te_feature.shape[0]*16).reshape(te_feature.shape[0],16)
a=np.hstack((te_feature,b)).reshape(te_feature.shape[0],12,12)
# 给数据增加一个维度，使数据和网络结构匹配
x_test = a.reshape(a.shape[0], 12, 12, 1)
#x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
print("x_train.shape", x_test.shape)
te_label=te_label[:,-1].astype(np.uint8)


x_train.shape (10964, 12, 12, 1)
x_train.shape (2741, 12, 12, 1)


In [32]:

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#from keras.layers.normalization import BatchNormalization

batch_size = 50
num_classes = 5
epochs = 100

#input image dimensions
input_shape = (12, 12, 1)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=input_shape,padding='SAME'))
model.add(MaxPooling2D((2, 2),padding='SAME'))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu',padding='SAME'))
model.add(MaxPooling2D(pool_size=(2, 2),padding='SAME'))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation='relu',padding='SAME'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 12, 12, 32)        320       
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 6, 6, 64)          18496     
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 3, 3, 64)          0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 3, 3, 64)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 3, 3, 128)        

In [33]:
history = model.fit(x_train, tr_label, batch_size=10, epochs=200, validation_data=(x_test, te_label))

Train on 10964 samples, validate on 2741 samples
Epoch 1/200
10964/10964 [==============================] - 7s 616us/sample - loss: 1.5266 - sparse_categorical_accuracy: 0.2974 - val_loss: 1.3798 - val_sparse_categorical_accuracy: 0.3907
Epoch 2/200
10964/10964 [==============================] - 6s 585us/sample - loss: 1.3650 - sparse_categorical_accuracy: 0.4122 - val_loss: 1.2811 - val_sparse_categorical_accuracy: 0.4721
Epoch 3/200
10964/10964 [==============================] - 6s 536us/sample - loss: 1.2670 - sparse_categorical_accuracy: 0.4697 - val_loss: 1.1337 - val_sparse_categorical_accuracy: 0.5465
Epoch 4/200
10964/10964 [==============================] - 6s 515us/sample - loss: 1.1610 - sparse_categorical_accuracy: 0.5265 - val_loss: 1.0195 - val_sparse_categorical_accuracy: 0.6056
Epoch 5/200
10964/10964 [==============================] - 6s 532us/sample - loss: 1.0892 - sparse_categorical_accuracy: 0.5702 - val_loss: 0.9400 - val_sparse_categorical_accuracy: 0.6330
Epoch 

Epoch 44/200
10964/10964 [==============================] - 6s 520us/sample - loss: 0.6428 - sparse_categorical_accuracy: 0.7491 - val_loss: 0.5011 - val_sparse_categorical_accuracy: 0.8136
Epoch 45/200
10964/10964 [==============================] - 6s 530us/sample - loss: 0.6379 - sparse_categorical_accuracy: 0.7484 - val_loss: 0.5035 - val_sparse_categorical_accuracy: 0.8103
Epoch 46/200
10964/10964 [==============================] - 6s 521us/sample - loss: 0.6372 - sparse_categorical_accuracy: 0.7534 - val_loss: 0.4896 - val_sparse_categorical_accuracy: 0.8081
Epoch 47/200
10964/10964 [==============================] - 6s 554us/sample - loss: 0.6346 - sparse_categorical_accuracy: 0.7547 - val_loss: 0.4974 - val_sparse_categorical_accuracy: 0.8081
Epoch 48/200
10964/10964 [==============================] - 6s 522us/sample - loss: 0.6312 - sparse_categorical_accuracy: 0.7511 - val_loss: 0.4838 - val_sparse_categorical_accuracy: 0.8125
Epoch 49/200
10964/10964 [========================

10964/10964 [==============================] - 6s 536us/sample - loss: 0.5559 - sparse_categorical_accuracy: 0.7827 - val_loss: 0.4436 - val_sparse_categorical_accuracy: 0.8169
Epoch 88/200
10964/10964 [==============================] - 6s 536us/sample - loss: 0.5617 - sparse_categorical_accuracy: 0.7769 - val_loss: 0.4469 - val_sparse_categorical_accuracy: 0.8117
Epoch 89/200
10964/10964 [==============================] - 6s 540us/sample - loss: 0.5555 - sparse_categorical_accuracy: 0.7866 - val_loss: 0.4358 - val_sparse_categorical_accuracy: 0.8278
Epoch 90/200
10964/10964 [==============================] - 6s 524us/sample - loss: 0.5525 - sparse_categorical_accuracy: 0.7811 - val_loss: 0.4390 - val_sparse_categorical_accuracy: 0.8179
Epoch 91/200
10964/10964 [==============================] - 6s 535us/sample - loss: 0.5530 - sparse_categorical_accuracy: 0.7828 - val_loss: 0.4260 - val_sparse_categorical_accuracy: 0.8391
Epoch 92/200
10964/10964 [==============================] - 6s 

10964/10964 [==============================] - 6s 534us/sample - loss: 0.5231 - sparse_categorical_accuracy: 0.7977 - val_loss: 0.3987 - val_sparse_categorical_accuracy: 0.8398
Epoch 131/200
10964/10964 [==============================] - 6s 568us/sample - loss: 0.5075 - sparse_categorical_accuracy: 0.7988 - val_loss: 0.4098 - val_sparse_categorical_accuracy: 0.8333
Epoch 132/200
10964/10964 [==============================] - 6s 562us/sample - loss: 0.5171 - sparse_categorical_accuracy: 0.7948 - val_loss: 0.4062 - val_sparse_categorical_accuracy: 0.8238
Epoch 133/200
10964/10964 [==============================] - 6s 548us/sample - loss: 0.5153 - sparse_categorical_accuracy: 0.8014 - val_loss: 0.3900 - val_sparse_categorical_accuracy: 0.8366
Epoch 134/200
10964/10964 [==============================] - 6s 552us/sample - loss: 0.5134 - sparse_categorical_accuracy: 0.7983 - val_loss: 0.3860 - val_sparse_categorical_accuracy: 0.8395
Epoch 135/200
10964/10964 [==============================] 

Epoch 173/200
10964/10964 [==============================] - 6s 556us/sample - loss: 0.5034 - sparse_categorical_accuracy: 0.8056 - val_loss: 0.3641 - val_sparse_categorical_accuracy: 0.8468
Epoch 174/200
10964/10964 [==============================] - 6s 528us/sample - loss: 0.4905 - sparse_categorical_accuracy: 0.8068 - val_loss: 0.3697 - val_sparse_categorical_accuracy: 0.8497
Epoch 175/200
10964/10964 [==============================] - 6s 553us/sample - loss: 0.4981 - sparse_categorical_accuracy: 0.8057 - val_loss: 0.3789 - val_sparse_categorical_accuracy: 0.8446
Epoch 176/200
10964/10964 [==============================] - 6s 536us/sample - loss: 0.4961 - sparse_categorical_accuracy: 0.8037 - val_loss: 0.3769 - val_sparse_categorical_accuracy: 0.8493
Epoch 177/200
10964/10964 [==============================] - 6s 555us/sample - loss: 0.4864 - sparse_categorical_accuracy: 0.8106 - val_loss: 0.3708 - val_sparse_categorical_accuracy: 0.8522
Epoch 178/200
10964/10964 [==================

In [56]:
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2688/2688 [==============================] - 0s 83us/sample - loss: 0.3008 - sparse_categorical_accuracy: 0.8955
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       542
           1       0.87      0.86      0.87       554
           2       0.84      0.97      0.90       536
           3       0.91      0.93      0.92       525
           4       0.96      0.71      0.82       531

    accuracy                           0.90      2688
   macro avg       0.90      0.90      0.89      2688
weighted avg       0.90      0.90      0.89      2688



In [60]:
#2
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2233/2233 [==============================] - 0s 81us/sample - loss: 0.5398 - sparse_categorical_accuracy: 0.7841
              precision    recall  f1-score   support

           0       0.80      0.98      0.88       462
           1       0.86      0.64      0.73       450
           2       0.73      0.85      0.78       450
           3       0.77      0.83      0.80       422
           4       0.78      0.62      0.69       449

    accuracy                           0.78      2233
   macro avg       0.79      0.78      0.78      2233
weighted avg       0.79      0.78      0.78      2233



In [64]:
#3
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2689/2689 [==============================] - 0s 96us/sample - loss: 0.3758 - sparse_categorical_accuracy: 0.8490
              precision    recall  f1-score   support

           0       0.86      1.00      0.92       529
           1       0.79      0.80      0.80       544
           2       0.85      0.88      0.86       549
           3       0.87      0.97      0.92       526
           4       0.89      0.61      0.72       541

    accuracy                           0.85      2689
   macro avg       0.85      0.85      0.84      2689
weighted avg       0.85      0.85      0.84      2689



In [68]:
#4
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2087/2087 [==============================] - 0s 81us/sample - loss: 0.4726 - sparse_categorical_accuracy: 0.8400
              precision    recall  f1-score   support

           0       0.89      0.97      0.93       424
           1       0.83      0.63      0.71       409
           2       0.80      0.89      0.85       424
           3       0.86      0.94      0.89       404
           4       0.82      0.77      0.79       426

    accuracy                           0.84      2087
   macro avg       0.84      0.84      0.83      2087
weighted avg       0.84      0.84      0.84      2087



In [72]:
#5
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2159/2159 [==============================] - 0s 80us/sample - loss: 0.5280 - sparse_categorical_accuracy: 0.7879
              precision    recall  f1-score   support

           0       0.84      1.00      0.91       413
           1       0.77      0.75      0.76       450
           2       0.77      0.79      0.78       438
           3       0.73      0.88      0.79       430
           4       0.88      0.52      0.65       428

    accuracy                           0.79      2159
   macro avg       0.80      0.79      0.78      2159
weighted avg       0.80      0.79      0.78      2159



In [76]:
#6
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2279/2279 [==============================] - 0s 78us/sample - loss: 0.4154 - sparse_categorical_accuracy: 0.8425
              precision    recall  f1-score   support

           0       0.91      0.99      0.95       447
           1       0.82      0.70      0.76       477
           2       0.79      0.96      0.87       445
           3       0.80      0.92      0.85       438
           4       0.92      0.67      0.77       472

    accuracy                           0.84      2279
   macro avg       0.85      0.85      0.84      2279
weighted avg       0.85      0.84      0.84      2279



In [30]:
#7
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2377/2377 [==============================] - 0s 79us/sample - loss: 0.4426 - sparse_categorical_accuracy: 0.8342
              precision    recall  f1-score   support

           0       0.96      1.00      0.98       489
           1       0.84      0.73      0.78       463
           2       0.75      0.89      0.82       488
           3       0.80      0.82      0.81       463
           4       0.82      0.71      0.76       474

    accuracy                           0.83      2377
   macro avg       0.84      0.83      0.83      2377
weighted avg       0.84      0.83      0.83      2377



In [89]:
#8
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2170/2170 [==============================] - 0s 118us/sample - loss: 0.4150 - sparse_categorical_accuracy: 0.8359
              precision    recall  f1-score   support

           0       0.87      0.97      0.92       443
           1       0.78      0.79      0.78       430
           2       0.83      0.88      0.85       438
           3       0.83      0.85      0.84       432
           4       0.87      0.69      0.77       427

    accuracy                           0.84      2170
   macro avg       0.84      0.83      0.83      2170
weighted avg       0.84      0.84      0.83      2170



In [6]:
#9
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2200/2200 [==============================] - 0s 110us/sample - loss: 0.5254 - sparse_categorical_accuracy: 0.7695
              precision    recall  f1-score   support

           0       0.71      0.97      0.82       458
           1       0.71      0.69      0.70       430
           2       0.81      0.81      0.81       425
           3       0.84      0.78      0.81       459
           4       0.81      0.58      0.68       428

    accuracy                           0.77      2200
   macro avg       0.78      0.77      0.76      2200
weighted avg       0.78      0.77      0.77      2200



In [85]:
#10
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2476/2476 [==============================] - 0s 79us/sample - loss: 0.4726 - sparse_categorical_accuracy: 0.8187
              precision    recall  f1-score   support

           0       0.81      0.99      0.89       503
           1       0.85      0.65      0.74       508
           2       0.78      0.91      0.84       474
           3       0.83      0.92      0.87       481
           4       0.84      0.64      0.72       510

    accuracy                           0.82      2476
   macro avg       0.82      0.82      0.81      2476
weighted avg       0.82      0.82      0.81      2476



In [10]:
#11
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2433/2433 [==============================] - 0s 78us/sample - loss: 0.4646 - sparse_categorical_accuracy: 0.8245
              precision    recall  f1-score   support

           0       0.93      1.00      0.96       499
           1       0.82      0.68      0.74       496
           2       0.85      0.86      0.85       482
           3       0.69      0.92      0.79       468
           4       0.88      0.67      0.76       488

    accuracy                           0.82      2433
   macro avg       0.83      0.83      0.82      2433
weighted avg       0.83      0.82      0.82      2433



In [14]:
#12
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

1974/1974 [==============================] - 0s 78us/sample - loss: 0.6383 - sparse_categorical_accuracy: 0.7604
              precision    recall  f1-score   support

           0       0.88      0.95      0.91       392
           1       0.72      0.59      0.65       390
           2       0.70      0.92      0.79       417
           3       0.76      0.85      0.80       381
           4       0.77      0.48      0.59       394

    accuracy                           0.76      1974
   macro avg       0.76      0.76      0.75      1974
weighted avg       0.76      0.76      0.75      1974



In [18]:
#13
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2741/2741 [==============================] - 0s 78us/sample - loss: 0.3540 - sparse_categorical_accuracy: 0.8581
              precision    recall  f1-score   support

           0       0.91      1.00      0.95       552
           1       0.82      0.84      0.83       515
           2       0.80      0.89      0.84       588
           3       0.88      0.90      0.89       542
           4       0.89      0.66      0.76       544

    accuracy                           0.86      2741
   macro avg       0.86      0.86      0.85      2741
weighted avg       0.86      0.86      0.85      2741



In [22]:
#14
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2440/2440 [==============================] - 0s 105us/sample - loss: 0.3961 - sparse_categorical_accuracy: 0.8582
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       505
           1       0.81      0.81      0.81       484
           2       0.82      0.91      0.86       501
           3       0.82      0.86      0.84       458
           4       0.87      0.70      0.78       492

    accuracy                           0.86      2440
   macro avg       0.86      0.86      0.86      2440
weighted avg       0.86      0.86      0.86      2440



In [26]:
#15
pred_y=model.predict(x_test, batch_size=100)
#返回索引
pred_yindex=np.argmax(pred_y, axis=1)
score = model.evaluate(x_test, te_label, batch_size=30)
print(sklearn.metrics.classification_report(te_label,pred_yindex))

2107/2107 [==============================] - 0s 89us/sample - loss: 0.4659 - sparse_categorical_accuracy: 0.8391
              precision    recall  f1-score   support

           0       0.90      0.91      0.91       438
           1       0.82      0.72      0.77       422
           2       0.82      0.96      0.88       415
           3       0.81      0.82      0.82       415
           4       0.83      0.79      0.81       417

    accuracy                           0.84      2107
   macro avg       0.84      0.84      0.84      2107
weighted avg       0.84      0.84      0.84      2107

